In [ ]:
import pandas as pd
from datetime import datetime
matches_2022 = pd.read_csv('data/matches_2022.csv')
matches_2022.head()

In [ ]:
# Show the columns of the dataframe in alphabetical order
sorted(matches_2022.columns)

In [ ]:
# Show the df with all columns except ones that contain 'loser' or 'winner'
match_cols = list(matches_2022[[col for col in matches_2022.columns if 'loser' not in col and 'winner' not in col]].columns)

# Print match_cols but without quotes around each column name
print(*match_cols, sep=', ')

In [ ]:
# Then return it as a list of dictionaries
matches_2022[[col for col in matches_2022.columns if 'loser' not in col and 'winner' not in col]].to_dict('records')

## Get unique players by name and ID

In [ ]:
# Get only the column names which include the substring 'name'
matches_2022.filter(regex='name').columns

In [ ]:
# Get only rows where the value of column 'winner_name' or 'loser_name' includes 'Tim van Rijthoven' without accounting for case
matches_2022[matches_2022['winner_name'].str.contains('Tim van Rijthoven', case=False) | matches_2022['loser_name'].str.contains('Tim van Rijthoven', case=False)]

# Same as above but only show the winner name or loser name columns
matches_2022[matches_2022['winner_name'].str.contains('Tim van Rijthoven', case=False) | matches_2022['loser_name'].str.contains('Tim van Rijthoven', case=False)][['winner_name', 'loser_name']]

In [ ]:
# Turn the above 2 lines into a function which outputs the index number of the found rows and the ratio of Tim's presence in the winner_name to loser_name columns
def find_matches_by_name(name):
    # Find the rows where name is present in the winner_name or loser_name columns
    name_matches_2022 = matches_2022[matches_2022['winner_name'].str.contains(name, case=False) | matches_2022['loser_name'].str.contains(name, case=False)]
    name_matches_index = name_matches_2022.index

    # Get the percent wins of the player
    won_percent = len(name_matches_2022[name_matches_2022['winner_name'].str.contains(name, case=False)]) / len(name_matches_2022)

    # From the latest row in the row index, get the value of the largest date value in the tourney_date column
    name_matches_latest_date = name_matches_2022.iloc[-1]['tourney_date']
    name_matches_latest_date = datetime.strptime(str(name_matches_latest_date), '%Y%m%d').strftime('%Y-%m-%d') # YYYY-MM-DD format

    # If the winner_name in the last row is equal to the name (not accounting for case), get that row's winner_id, otherwise get the loser_id
    if name_matches_2022.iloc[-1]['winner_name'].lower() == name.lower():
        id = name_matches_2022.iloc[-1]['winner_id']
    else:
        id = name_matches_2022.iloc[-1]['loser_id']

    # As a dictionary, return the name of the player, a list of the index numbers of the rows where the player is present and the ratio of the player's presence in the winner_name to loser_name columns
    return {'name': name, 'id': id, 'matches_2022': [name_matches_index], 'win_percent': won_percent, 'latest_date': name_matches_latest_date}

# Get the index number of the rows where Tim is present and the ratio of Tim's presence in the winner_name to loser_name columns
find_matches_by_name('Tim van Rijthoven')


In [ ]:
def find_matches_by_id(id):
    # Find the rows where name is present in the winner_name or loser_name columns
    id_won_matches_2022 = matches_2022[matches_2022['winner_id'] == id]
    id_lost_matches_2022 = matches_2022[matches_2022['loser_id'] == id]
    id_matches_2022 = pd.concat([id_won_matches_2022, id_lost_matches_2022])
    
    id_matches_index = id_matches_2022.index

    # Get the percent wins of the player
    won_percent = len(id_won_matches_2022) / len(id_matches_2022)
    
    # From the latest row in the row index, get the value of the largest date value in the tourney_date column
    id_matches_latest_date = id_matches_2022.iloc[-1]['tourney_date']
    id_matches_latest_date = datetime.strptime(str(id_matches_latest_date), '%Y%m%d').strftime('%Y-%m-%d') # YYYY-MM-DD format

    # Get the name of the player
    name = id_matches_2022.iloc[-1]['winner_name'] if id_matches_2022.iloc[-1]['winner_id'] == id else id_matches_2022.iloc[-1]['loser_name']

    # As a dictionary, return the name of the player, a list of the index numbers of the rows where the player is present and the ratio of the player's presence in the winner_name to loser_name columns
    return {'name': name, 'id': id, 'matches_2022': [id_matches_index], 'win_percent': won_percent, 'latest_date': id_matches_latest_date}

# find_matches_by_id(126646)

In [ ]:
# Get the unique ids of the players
unique_ids = set(list(matches_2022['winner_id'].unique()) + list(matches_2022['loser_id'].unique()))

# Now make this performance even better by using multiprocessing
import multiprocessing as mp

pool = mp.Pool(processes=4)
all_people = pool.map(find_matches_by_id, unique_ids)
pool.close()

# Turn the list of dictionaries into a dataframe
all_people_df = pd.DataFrame(all_people)

# # Sort the dataframe by the win/lose ratio
# all_matches_matches_2022.sort_values(by='win_lose', ascending=False)

# # Get the top 10 players with the highest win/lose ratio
# all_matches_matches_2022.sort_values(by='win_lose', ascending=False).head(10)

all_people_df

## Get more characteristics

In [ ]:
# Let's use Rafael Nadal as an example, because I've heard his name before. Get just his id number
nadal_id = all_people_df[all_people_df['name'].str.contains('Nadal', case=False)]['id'].values[0]

nadal = find_matches_by_id(nadal_id)
nadal

In [ ]:
# Get rows from the matches_2022 dataframe based on gilles['matches_2022']
nadal_df = matches_2022.iloc[nadal['matches_2022'][0]]
nadal_df.shape

In [ ]:
# Get list of columns from nadal_df
nadal_df.columns

Columns include information about the tournament, the winner and the loser, and more. Using the dictionary text file in the source Github repo (found here: https://github.com/JeffSackmann/tennis_atp/blob/master/matches_data_dictionary.txt), here are some explanations of the columns.
> - draw_size: number of players in the draw, often rounded up to the nearest power of 2. (For instance, a tournament with 28 players may be shown as 32.)
> - tourney_level: 
>   - For men: 'G' = Grand Slams, 'M' = Masters 1000s, 'A' = other tour-level events, 'C' = Challengers, 'S' = Satellites/ITFs, 'F' = Tour finals and other season-ending events, and 'D' = Davis Cup 
>   - For women, there are several additional tourney_level codes, including 'P' = Premier, 'PM' = Premier Mandatory, and 'I' = International. The various levels of ITFs are given by the prize money (in thousands), such as '15' = ITF $15,000. Other codes, such as 'T1' for Tier I (and so on) are used for older WTA tournament designations. 'D' is used for Federation/Fed/Billie Jean King Cup, and also for Wightman Cup and Bonne Bell Cup.
>   - Others, eventually for both genders: 'E' = exhibition (events not sanctioned by the tour, though the definitions can be ambiguous), 'J' = juniors, and 'T' = team tennis, which does yet appear anywhere in the dataset but will at some point.
> - match_num: somewhat arbitrary
> - winner_entry: 'WC' = wild card, 'Q' = qualifier, 'LL' = lucky loser, 'PR' = protected ranking, 'ITF' = ITF entry, and there are a few others that are occasionally used.
> - winner_hand: R = right, L = left, U = unknown. For ambidextrous players, this is their serving hand.
> - winner_ht: height in centimeters, where available
> - winner_ioc: three-character country code
> - winner_age: age, in years, as of the tourney_date
> - best_of: '3' or '5', indicating the the number of sets for this match
> - minutes: match length

Possibly interesting filters are 
1. Which tournaments has this person participated in?
2. The tournament levels this person has participated in
3. Main hand
4. Country they're from
5. Age
6. How many sets they play on average
7. Average game length, or longest, shortest games
8. Which surface they're strongest on (win percentage they have on that surface)

In [ ]:
# From nadal_df, find row where the loser name is 'Daniil Medvedev' and the tourney_name is 'Australian Open'
nadal_medvedev = nadal_df[(nadal_df['loser_name'] == 'Daniil Medvedev') & (nadal_df['tourney_name'] == 'Australian Open')][['winner_name', 
  'loser_name', 'tourney_name', 'tourney_level', 'tourney_date',
  'draw_size', 'score', 'round',
  'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon', 'w_SvGms', 'w_bpSaved', 'w_bpFaced',
  'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon', 'l_SvGms', 'l_bpSaved', 'l_bpFaced']]

# Print the row in a nice to read format, with column-value pairs on separate lines
for col in nadal_medvedev.columns:
    print(col, ':', nadal_medvedev[col].values[0])


In [ ]:
def make_person(id):
    # From all people, get the row where the winner_id is equal to the id
    index_2022 = find_matches_by_id(id)['matches_2022']

    # Get the rows from the matches_2022 dataframe based on the index, split into won and lost
    matches = matches_2022.iloc[index_2022[0]]
    won_matches = matches.loc[matches_2022['winner_id'] == id]
    lost_matches = matches.loc[matches_2022['loser_id'] == id]
    
    # Create a dictionary of lists of wanted properties (same as above), using the won_matches and lost_matches dataframes
    person = {
        'name': won_matches.iloc[-1]['winner_name'] if won_matches.iloc[-1]['winner_id'] == id else lost_matches.iloc[-1]['loser_name'],
        'id': id,
        'rank': won_matches.iloc[-1]['winner_rank'] if won_matches.iloc[-1]['winner_id'] == id else lost_matches.iloc[-1]['loser_rank'],
        'rank_points': won_matches.iloc[-1]['winner_rank_points'] if won_matches.iloc[-1]['winner_id'] == id else lost_matches.iloc[-1]['loser_rank_points'],
        'matches_2022': index_2022,
        'won_matches_2022': len(won_matches),
        'lost_matches_2022': len(lost_matches),
        'win_percent': len(won_matches) / len(matches),
        'latest_date': matches.iloc[-1]['tourney_date'],
        'tourney_names': list(matches['tourney_name'].unique()),
        # summed stats
        'sum_aces': won_matches['w_ace'].sum() + lost_matches['l_ace'].sum(),
        'sum_double_faults': won_matches['w_df'].sum() + lost_matches['l_df'].sum(),
        'sum_service_points': won_matches['w_svpt'].sum() + lost_matches['l_svpt'].sum(),
        'sum_first_serve_points': won_matches['w_1stIn'].sum() + lost_matches['l_1stIn'].sum(),
        'sum_first_serve_points_won': won_matches['w_1stWon'].sum() + lost_matches['l_1stWon'].sum(),
        'sum_second_serve_points_won': won_matches['w_2ndWon'].sum() + lost_matches['l_2ndWon'].sum(),
        'sum_serve_games': won_matches['w_SvGms'].sum() + lost_matches['l_SvGms'].sum(),
        'sum_break_points_saved': won_matches['w_bpSaved'].sum() + lost_matches['l_bpSaved'].sum(),
        'sum_break_points_faced': won_matches['w_bpFaced'].sum() + lost_matches['l_bpFaced'].sum(),
        # average stats
        'avg_aces': (won_matches['w_ace'].sum() + lost_matches['l_ace'].sum()) / len(matches),
        'avg_double_faults': (won_matches['w_df'].sum() + lost_matches['l_df'].sum()) / len(matches),
        'avg_service_points': (won_matches['w_svpt'].sum() + lost_matches['l_svpt'].sum()) / len(matches),
        'avg_first_serve_points': (won_matches['w_1stIn'].sum() + lost_matches['l_1stIn'].sum()) / len(matches),
        'avg_first_serve_points_won': (won_matches['w_1stWon'].sum() + lost_matches['l_1stWon'].sum()) / len(matches),
        'avg_second_serve_points_won': (won_matches['w_2ndWon'].sum() + lost_matches['l_2ndWon'].sum()) / len(matches),
        'avg_serve_games': (won_matches['w_SvGms'].sum() + lost_matches['l_SvGms'].sum()) / len(matches),
        'avg_break_points_saved': (won_matches['w_bpSaved'].sum() + lost_matches['l_bpSaved'].sum()) / len(matches),
        'avg_break_points_faced': (won_matches['w_bpFaced'].sum() + lost_matches['l_bpFaced'].sum()) / len(matches)
    }
    
    return person

make_person(104745)

In [ ]:
# TODO: import all the csv files from the past 10 years and then do the same thing as above. Can just add the new csv file names to the functions above and then run the functions again.


## Add data to DB


In [1]:
from database_workers.item_handling.tennis_editor import TennisEditor
from datetime import datetime
import pandas as pd
matches_2022 = pd.read_csv('data/matches_2022.csv')

# Create a TennisEditor object, use matches_2022 df as the database
editor = TennisEditor(data=matches_2022)

# Add players to db
editor.upload_players_to_db()


  5%|▍         | 20/408 [00:00<00:03, 102.43it/s]

{'name': 'Nicolas Mejia', 'player_id': 200711} has been created
{'name': 'Dennis Novak', 'player_id': 110602} has been created
{'name': 'Gilles Simon', 'player_id': 104468} has been created
{'name': 'Kasidit Samrej', 'player_id': 208937} has been created
{'name': 'Brandon Nakashima', 'player_id': 206909} has been created
{'name': 'Stan Wawrinka', 'player_id': 104527} has been created
{'name': 'Jo-Wilfried Tsonga', 'player_id': 104542} has been created
{'name': 'John Isner', 'player_id': 104545} has been created
{'name': 'Lukas Rosol', 'player_id': 104586} has been created
{'name': 'Nikola Milojevic', 'player_id': 110748} has been created
{'name': 'Yosuke Watanuki', 'player_id': 133297} has been created
{'name': 'Daniel Altmaier', 'player_id': 127157} has been created
{'name': 'Blaise Bicknell', 'player_id': 209080} has been created
{'name': 'Hamad Medjedovic', 'player_id': 209098} has been created
{'name': 'Pablo Cuevas', 'player_id': 104655} has been created
{'name': 'Pablo Andujar', 

 16%|█▌        | 66/408 [00:00<00:01, 183.88it/s]

{'name': 'Fitriadi M Rifqi', 'player_id': 207134} has been created
{'name': 'Alexander Zverev', 'player_id': 100644} has been created
{'name': 'Rafael Nadal', 'player_id': 104745} has been created
{'name': 'Richard Gasquet', 'player_id': 104755} has been created
{'name': 'Denis Shapovalov', 'player_id': 133430} has been created
{'name': 'Filip Misolic', 'player_id': 209209} has been created
{'name': 'Emilio Nava', 'player_id': 207182} has been created
{'name': 'Gael Monfils', 'player_id': 104792} has been created
{'name': 'Denis Istomin', 'player_id': 104797} has been created
{'name': 'Borna Gojo', 'player_id': 127339} has been created
{'name': 'Duje Ajdukovic', 'player_id': 207213} has been created
{'name': 'Elmer Moller', 'player_id': 209284} has been created
{'name': 'Andres Martin', 'player_id': 211346} has been created
{'name': 'Illia Biloborodko', 'player_id': 207277} has been created
{'name': 'Robin Haase', 'player_id': 104898} has been created
{'name': 'Nicolas Barrientos', 'pl

 27%|██▋       | 109/408 [00:00<00:01, 197.19it/s]

...

{'name': 'Alexandar Lazarov', 'player_id': 200247} has been created
{'name': 'Diego Schwartzman', 'player_id': 106043} has been created
{'name': 'Bernabe Zapata Miralles', 'player_id': 126523} has been created
{'name': 'Denis Kudla', 'player_id': 106045} has been created
{'name': 'Nicolas Kicker', 'player_id': 106044} has been created
{'name': 'Carlos Taberner', 'player_id': 126535} has been created
{'name': 'Facundo Mena', 'player_id': 106057} has been created
{'name': 'Jack Sock', 'player_id': 106058} has been created
{'name': 'Zizou Bergs', 'player_id': 200267} has been created
{'name': 'Geoffrey Blancaneaux', 'player_id': 144972} has been created
{'name': 'Santiago Fa Rodriguez Taverna', 'player_id': 144973} has been created
{'name': 'Nuno Borges', 'player_id': 132686} has been created
{'name': 'Beibit Zhukayev', 'player_id': 202320} has been created
{'name': 'Marco Cecchinato', 'player_id': 106065} has been created
{'name': 'Hady Habib', 'player_id': 200273} has been created
{'nam

 89%|████████▉ | 365/408 [00:01<00:00, 241.72it/s]

{'name': 'Hugo Gaston', 'player_id': 200384} has been created
{'name': 'Kacper Zuk', 'player_id': 200390} has been created
{'name': 'Juan Pablo Ficovich', 'player_id': 126663} has been created
{'name': 'Jason Kubler', 'player_id': 106186} has been created
{'name': 'Tallon Griekspoor', 'player_id': 134868} has been created
{'name': 'Hugo Dellien', 'player_id': 106198} has been created
{'name': 'Jiri Vesely', 'player_id': 106210} has been created
{'name': 'Oscar Otte', 'player_id': 106214} has been created
{'name': 'Bjorn Fratangelo', 'player_id': 106216} has been created
{'name': 'Marcos Giron', 'player_id': 106218} has been created
{'name': 'Philip Henning', 'player_id': 202475} has been created
{'name': 'Dimitar Kuzmanov', 'player_id': 106220} has been created
{'name': 'Aleksandre Metreveli', 'player_id': 106223} has been created
{'name': 'Ji Sung Nam', 'player_id': 106227} has been created
{'name': 'Juan Ignacio Londero', 'player_id': 106228} has been created
{'name': 'Zsombor Piros'

100%|██████████| 408/408 [00:01<00:00, 207.79it/s]

{'name': 'Brandon Holt', 'player_id': 200587} has been created
{'name': 'Gonzalo Villanueva', 'player_id': 106380} has been created
{'name': 'Michail Pervolarakis', 'player_id': 133018} has been created
{'name': 'Giulio Zeppieri', 'player_id': 206748} has been created
{'name': 'Pedro Cachin', 'player_id': 106398} has been created
{'name': 'Nick Kyrgios', 'player_id': 106401} has been created
{'name': 'Elliot Benchetrit', 'player_id': 200611} has been created
{'name': 'Alexei Popyrin', 'player_id': 200615} has been created
{'name': 'Yoshihito Nishioka', 'player_id': 106415} has been created
{'name': 'Sebastian Korda', 'player_id': 200624} has been created
{'name': 'Max Hans Rehberg', 'player_id': 208819} has been created
{'name': 'Daniil Medvedev', 'player_id': 106421} has been created
{'name': 'Thanasi Kokkinakis', 'player_id': 106423} has been created
{'name': 'Cristian Garin', 'player_id': 106426} has been created
{'name': 'Borna Coric', 'player_id': 106432} has been created
{'name':

In [2]:
editor.find_matches_by_id(104586)

{'name': 'Lukas Rosol',
 'id': 104586,
 'matches': [Index([1548, 2534], dtype='int64')]}